# 物件探しも自動化！ ArcGIS API for Python で近隣検索ツールの開発

## はじめに
 米国Esri社では、開発者向け製品として [`ArcGIS Developer`](https://www.esrij.com/products/arcgis-for-developers/) がリリースされており、[`ArcGIS API for Python`](https://www.esrij.com/products/arcgis-api-for-python/) もその一つとして提供しています。これは、 WebGIS での空間データの管理・操作に用いられることが多い API で、主に GIS と連動した空間データの処理の自動化に用いられるなどしています。また、`Python` という言語の豊富なライブラリを用いてそれらのデータのビジュアライズや分析を行なうことに長けているのが特長です。
 
 このサンプルノートブックは、`ArcGIS API for Python` で使用できるモジュールを用いて、ある地点の近くにある道路の幅員と 500m 圏内にある指定した建物情報の取得を実現しています。これらの結果は、`CSV` に書き込まれます。
検索する場所の指定の方法として、以下の二つの方法を用意しています。
 
 
 ① `Jupyter Notebook` 上に表示したマップをクリックする
 
 ②住所の情報が記載された `CSV` からジオコーディングを行う
 
 

## ツールの実装について
ここからは、`arcgis` モジュールをどのように使用して近隣検索ツールを実現しているかを紹介します。

道路に関するデータは、ESRIジャパンで販売している [ArcGIS Geo Suite 道路網データ](https://www.esrij.com/products/data-content-geosuite-douromo/) を使用しています。

In [1]:
import arcgis # arcgis api for python を使うため
import pandas as pd # CSV からデータを取得するために使用
import numpy as np # distance 関数を使うために使用
from arcgis.gis import GIS # gis モジュールの使用
from arcgis.geocoding import geocode # geocoding で住所を座標に変換するために使用
from arcgis.geometry import Point, Polyline, Polygon, SpatialReference , distance, buffer # 主にジオメトリ変換に使うため

gis = GIS()

maps = gis.map("東京都千代田区麹町２丁目４−２０" ,16)
maps.basemap='streets-relief-vector' # basemap をベクター タイルに変更

In [2]:
# 道路網データ(フィーチャクラス)を入れる

sdf = pd.DataFrame.spatial.from_featureclass("<参照するファイル>")

## 近くにある道路の幅員を取得
今回は、任意の地点から最も近い道路の幅員を取得するようにしています。上記で入力した道路データ全てから一番近い道路はどれか探そうとすると時間がかかってしまうため、任意の地点を中心として [`geometry`](https://developers.arcgis.com/python/api-reference/arcgis.geometry.html) モジュールの [`buffer()`](https://developers.arcgis.com/python/api-reference/arcgis.geometry.html#buffer) メソッドを使用し、 50m バッファーを作成し、その中に含まれる道路のポリラインを `geometry` モジュールの [`intersect()`](https://developers.arcgis.com/python/api-reference/arcgis.geometry.html?#arcgis.geometry.Geometry.intersect) メソッドで抽出し、その中から道路の幅員を取得するようにしています。この時、 50m 圏内に道路がなければ 50m を足して、再び同様の処理を行います。バッファー内に道路のポリラインが複数含まれていて、それぞれ異なる幅員となっている場合、`geometry` モジュールの [`distance()`](https://developers.arcgis.com/python/api-reference/arcgis.geometry.html?#arcgis.geometry.distance) メソッドを使用し、任意の地点に最も近い道路の幅員を参照しています。この時、浮動小数点以下の値でも比較するために Python の [`decimal`](https://docs.python.org/ja/3/library/decimal.html) モジュールを使用しています。

これらによって得られた結果は、 [`widget`](https://developers.arcgis.com/python/api-reference/arcgis.widgets.html) モジュールの [`draw()`](https://developers.arcgis.com/python/api-reference/arcgis.widgets.html?#arcgis.widgets.MapView.draw) メソッドでマップに描画され、バッファーをクリックするとポップアップで道路の幅員が出力されます。それらの処理をここでは `nearby_road()` として関数化しています。

In [3]:
# 近くの道路の幅を取得して、 buffer を描画する

def nearby_road(point,gis_class,gs_road,map_name,place_name):
    """
    point:近隣検索を行う物件などの ジオメトリ型の Point , 
    gis_class:gis モジュールの GIS クラスを指定.
    road_data:geo suite 道路網データ, 
    map_name: gis.map() を実行している変数,
    place_name: 場所の名前等
    """
    widthid={'1': "13m以上", '2': "5.5m以上13.0ｍ未満", '3': "3.0m以上5.5m未満" , '4': "3.0m未満"} # 幅員の番号とその意味を示した dict 型配列

    buffer_dist=50
    while True: 
        point_buffer = buffer( geometries =[point],
                              in_sr=3857, 
                              distances=buffer_dist, 
                              unit="9001", 
                              out_sr=3857, 
                              buffer_sr=3857, 
                              geodesic=True,  
                              gis=gis_class)
        
        # list 型なので Polygon に変換
        point_buffer=Polygon(point_buffer[0]) 
        in_buff={}
        for i,road in enumerate(road_data["SHAPE"]) :
            if point_buffer.intersect(second_geometry=road,dimension=2): # buffer 内の polyline が重なっているかを検索
                in_buff[i]=point_buffer.intersect(second_geometry=road,dimension=2)
        
        # 50m の範囲内になかったら+50mして再計測
        if in_buff=={}: 
            buffer_dist+=50
            continue
        else:
            break
            
    width=[] # 道路の幅員番号を入れる箱
    for keyid in in_buff:
        width+=road_data["<フィールド名>"][keyid]
       
    
    # 以下すべての道路幅が同じなら終わり

    if len(width)==width.count(width[0]):
        road_width=width[0]
    
    else :
       
        # ここから下は道路幅が異なるなら実行　

        from decimal import Decimal # 浮動小数点での比較を行えるようにする

        for i,keyid in enumerate(in_buff): 
            
            dist2road = distance(spatial_ref=3857, 
                         geometry1=in_buff[keyid], 
                         geometry2=point, 
                         distance_unit='9001', 
                         geodesic=True, 
                         gis=gis_class
                     )
            
            if i==0 : 
                roadid=keyid 
                dist=Decimal(dist2road["distance"]) 
            if 0<i and Decimal(dist2road["distance"])<dis: # python は浮動小数点だと比較できないため decimal モジュールを使用
                roadid=keyid
                dist=Decimal(dist2road["distance"])

        
        road_width=road_data["<フィールド名>"][roadid] 
        
    # マップ上に道路までの baffer を描画する
    map_name.draw(point_buffer,
                popup= {'title':"最も近い道路の幅員",'content': place_name+"に最も近い道路は、幅員が" + widthid[road_width] + "である"},
                symbol = {
                      "type" : "esriSFS",
                      "style" : "esriSFSDiagonalCross",
                      "outline" : "blue",
                      "color": "lightblue"
                    }
                 ) 
    
    return widthid[road_width] # 道路幅のIDから幅を取得できるようにする

## POI を指定した近隣検索
[`geocording`](https://developers.arcgis.com/python/api-reference/arcgis.geocoding.html) モジュールの [`geocode()`](https://developers.arcgis.com/python/api-reference/arcgis.geocoding.html?#arcgis.geocoding.geocode) メソッドを使用して、任意の地点から 500m 圏内に特定の [`POI`](https://www.esrij.com/gis-guide/gis-other/point-of-interest/) があるかどうかを判定します。`geocode()` メソッドでは、建物の種別を指定すると任意の地点から近隣検索を行い、その建物の種別に関連する場所の情報を返します。ただし、その距離は指定できないため、任意の地点から `buffer()` メソッドで 500m バッファーを作成し、その中に目標物が存在しているかを `intersect()` で取得します。これらの処理をここでは `extent_500()` として関数化しています。

In [45]:
# 500m圏内で近隣検索

def extent_500(point,poi,gis_class,place_name,map_name): 
    """ 
    point:調査地点, 
    poi: POI として設定した施設種別, 
    gis_class: gis モジュールの GIS クラスを指定,
    place_name:場所の名前, 
    map_name: gis.map() の変数
    """
   
    poi_search = geocode(poi, location=point, out_sr=3857) 
    
    # 500mバッファーの作成
    point_buffer=buffer( geometries = [point], 
                            in_sr=3857, 
                            distances=500, 
                            unit="9001", 
                            out_sr=3857, 
                            buffer_sr=3857, 
                            geodesic=True, 
                            gis=gis_class)
    point_buffer=Polygon(point_buffer[0])
    
    map_name.draw(point_buffer) 
    
    poi_name=[]
    for address in poi_search:
        to_point=Point({'spatialReference' :{'latestWkid': 3857, 'wkid': 102100},'x': address['location']['x'] ,'y':  address['location']['y']})
        
        # buffer の x 属性に値がないとき
        if point_buffer.intersect(second_geometry=to_point,dimension=1).x!="NaN": 
            
            print(address['attributes']['Match_addr'])
            map_name.draw(to_point,
                      popup= {'title':address['attributes']['Match_addr'],
                              'content': "建物種別:"+ address['attributes']['Type'] + "<br>どこから500m圏内:" + place_name},
                              symbol = {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Cartographic/esriCartographyMarker_70_Yellow.png","contentType":"image/png","width":14,"height":14}
                         )
            poi_name.append(address['attributes']['Match_addr'])
    
    return poi_name # POIの目標物の名前を返す

## 実行結果を CSV に入力
実行結果を [`pandas`](https://pandas.pydata.org/pandas-docs/stable/index.html) の [`to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html?#pandas.DataFrame.to_csv) メソッドを使用して `CSV` に書き込みます。この時、入力する値は辞書型から取得していくようにしています。また、新たにファイルを作成する際には `addFlag` パラメーターに `True` を、既存のファイルに追加する際には `False` を指定します。新規作成の場合、辞書型のデータを一度 `DataFrame` に変換した後、 `CSV` に書き込んでいます。

なお、今回はジオコーディングを実行した値は保存しません。ジオコーディングした結果は、 `geocode()` メソッドの `forstrage` パラメーターを `True` にすることでのみ結果の保存を許されています。詳しくは[こちらのドキュメント](https://github.com/EsriJapan/arcgis-python-api/blob/master/guide/09-finding-places-with-geocoding/understanding-the-geocode-function_ja.ipynb)をご確認ください。

In [5]:
# CSVに入力

def make_csv(csv,dic,poi_name,addFlag): 
    """
    csv:書き込む csv ファイルの指定 
    dic:結果を辞書型で保存したもの 
    poiname: POI の施設種別 
    addFlag: False なら新規作成(上書き)
    """
    # 追加か新規作成かで分岐
    if(addFlag):
        # excelでの標準文字エンコードがshift_jisであるため、指定。
        dfs = pd.read_csv(csv, index_col=False, encoding='shift_jis') 
        dfs[poi_name] =[dic[i][poi] for i in range(len(dic)) if i<len(dic)] 
        dfs.to_csv(csv, encoding='shift_jis', index=False) 
    else:
        df = pd.DataFrame.from_dict(dic, orient='index') # orient はindexの方向で保存
        df.to_csv(csv, encoding='shift_jis', index=False) 


## onclick() による近隣検索
`widgets` モジュールの [`onclick()`](https://developers.arcgis.com/python/api-reference/arcgis.widgets.html?highlight=on_click#arcgis.widgets.MapView.on_click) メソッドからコールバック関数として返される座標を使用し、上記した関数と組み合わせた `nearby_info()` を関数化し、道路の幅員と指定した建物の近隣検索を行います。その結果は、`draw()` メソッドでマップに描画されます。また、近隣検索によって表示される建物に関してはその施設の名前が表示されるようになっています。
マップへの追加が終わったら次のセルで `onclick()` メソッドの `remove` パラメーターを `True` にし、同時に `make_csv()` を実行しています。 

In [42]:
# クリックした地点の周辺の情報を取得

results={}
i=0
def nearby_info(m, g):
    global poi
    poi="hospital"
    place_name="物件A" #変数名直す
    try:
        points = Point({'spatialReference' :{'latestWkid': g['spatialReference']['latestWkid'], 'wkid': g['spatialReference']['wkid']},'x': g['x'] ,'y':  g['y']})
        near_road=nearby_road(points,gis,sdf,maps,place_name)
        within=extent_500(points,poi,gis,place_name,maps) #変数名
        m.draw(g,
              symbol={"angle":0,"xoffset":2,"yoffset":8,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/LightBlueShinyPin.png","contentType":"image/png","width":14,"height":14})
        print(exfiv) # 500m 圏内にある目標物の名前を表示
        # symbol は,https://esri.github.io/arcgis-python-api/tools/symbol.html で作成可能. 
        if results!={}:
            i=len(results)            
            results[i]={"物件名": place_name ,"lat": g["x"],"lng": g["y"],"幅員":near_road, poi :len(within)}
           
        else:
            i=0            
            results[i]={"物件名": place_name ,"lat": g["x"],"lng": g["y"],"幅員":near_road, poi :len(within)}
            
        
    except:
        print("位置情報取得失敗") # エラー時

maps.on_click(nearby_info)


maps

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)

In [9]:
# コールバック関数の削除。これでマップへの追加をやめる。

maps.on_click(nearby_info,True)

onclick_file='./csv_folder/test_click.csv'
#make_csv(onclick_file,results,poi,False) # 新規作成になるため False で指定

![on_click demo](./image/nearby.gif)

In [27]:
# nearby_info() で作成したcsvを読みとり
    
onclick_csv=pd.read_csv(onclick_file,encoding="shift_jis")
onclick_csv

,物件名,lat,lng,幅員,hospital,park
0,物件A,15555755.96,4257287.26,5.5m以上13.0ｍ未満,3,2


## 近隣検索の結果を CSV に追加
`nearby_info()` による近隣検索の結果に他の `POI` による検索結果を追加するときに用います。これには `pandas` の [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) と近隣検索の関数 `extent_500()` を利用しています。

In [46]:
# nearby_info() で作ったファイルに追加する

poi="park"
result={}
for i,latlng in enumerate(onclick_csv.values): 
    csv_point=Point({'spatialReference' :{'latestWkid': 3857, 'wkid': 102100},'x': latlng[1] ,'y':  latlng[2]})
    within=extent_500(csv_point,poi,gis,latlng[0],maps) 
    result[i]={"物件名": latlng[0], "lat": latlng[1] , "lng": latlng[2], "幅員":latlng[3], poi :len(within)}
    
make_csv(onclick_file,result,poi,True) # POI を検索した結果を追加するだけなのでここは True で固定

清水谷公園
千鳥ヶ淵公園


![add_park](./image/add_search_park.png)

## 住所カラムのある CSV を読み取って近隣検索
以下では、`pandas` の [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) を使用して、 `CSV` を読み取ります。その中から住所に関するカラムを取得し、`geocode()` メソッドでジオコーディングを実行しています。ジオコーディングの結果の値を使って `extent_500()` で近隣検索と `nearby_road()` で一番近い道路の幅員を取得します。また、それらの結果を別の `CSV` に書き込みます。

In [14]:
# 住所に関する情報が書かれている CSV を取得

df_csv = pd.read_csv('./tokyo_sample.csv',encoding="shift_jis")
df_csv

,住所,階数
0,東京都千代田区平河町2-7-1,7階建て
1,東京都千代田区千代田１-１,4階建て
2,東京都千代田区北の丸公園３-１,2階建て


In [51]:
# 住所情報のある CSV から近隣検索

result={}
poi="park"

for i,address in enumerate(df_csv['住所']):
    # csv の一つのカラムからジオコード
    csv_geocode=geocode(address, out_sr=3857)[0] 
    csv_point=Point({'spatialReference' :{'latestWkid': 3857, 'wkid': 102100},'x': csv_geocode['location']['x'] ,'y':  csv_geocode['location']['y']})
    within=extent_500(csv_point,poi,gis,address,maps) 
    print(within)
    road_width=nearby_road(csv_point,gis,sdf,maps,address)
    maps.draw(csv_point,symbol={"angle":0,"xoffset":2,"yoffset":8,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/LightBlueShinyPin.png","contentType":"image/png","width":14,"height":14})    
    # ジオコードの結果を保存するには forstrage が必要なので、何件あったかだけ表示
    result[i]={"住所": address,"幅員":road_width, poi :len(within)} 
        
make_csv(testfile,result,poi,False)

清水谷公園
['清水谷公園']
皇居外苑
['皇居外苑']
北の丸公園
二の丸庭園
['北の丸公園', '二の丸庭園']


![csv2geocode](./image/csv_search_park.png)

In [52]:
# 出力先の csv を確認する
testfile='./csv_folder/tokyo_test.csv'

test = pd.read_csv(testfile,encoding="shift_jis")
test.head()

,住所,幅員,park
0,東京都千代田区平河町2-7-1,5.5m以上13.0ｍ未満,1
1,東京都千代田区千代田１-１,5.5m以上13.0ｍ未満,1
2,東京都千代田区北の丸公園３-１,3.0m以上5.5m未満,2
